In [2]:
using JuMP, CPLEX
import DataFrames
import HiGHS
import Plots
import SparseArrays
using Random
import Test  #src
function calculate_arc_cost(points::Vector{Vector{Int}})
    num_points = length(points)
    arc_cost = Array{Float64}(undef, num_points, num_points)
    for i in 1:num_points
        for j in 1:num_points
            if i == j
                arc_cost[i, j] = 100000.0  # Large value for self-loops
            elseif i in satellites && PI[i-1]==0 && j in satellites && PI[j-1]==0
                arc_cost[i, j] = 100000.0 
            else
                arc_cost[i, j] = sqrt(sum((points[i][k] - points[j][k])^2 for k in 1:length(points[i])))
            end
        end
    end
    for i in satellites
        if PI[i-1] == 0
            arc_cost[1,i] = 100000.0
        end
    end
    # for i in 1:num_points
    #     for j in 1:num_points
    #         print("  d[$i $j]=", round(arc_cost[i,j], digits=2))
    #     end
    #     println("")
    # end
    return arc_cost
end

struct Route
    cost::Float64
    sequence::Vector{Int}
    length::Int
    load::Int
 end
 
 function generateRoute(route::Vector{Int})
    cost = 0
    load = 0
    for i in 1:(length(route) - 1)
        if i!= 1
            load += demands[route[i]]
        end
        from_node = route[i]
        to_node = route[i + 1]
        cost+= arc_cost[from_node, to_node]
    end

    # println("Route:",Route(cost,route,length(route),load) )
    return Route(cost,route,length(route),load) 
 end

# Function to calculate the total demand for a path
function calculate_load(path, demands)
    load = 0
    for node in path[2:end-1]  # Exclude the start and end warehouses
        load += demands[node]
    end
    return load
end

# Function to generate all paths with multiple customers
function generate_paths(warehouses, customers, demands, max_load)
    paths = []
    # Generate all permutations of customer subsets (excluding empty set)
    for num_customers in 1:length(customers)
        for customer_subset in combinations(customers, num_customers)
            for customer_permutation in permutations(customer_subset)
                for start_warehouse in warehouses
                    for end_warehouse in warehouses
                        path = vcat([start_warehouse], customer_permutation, [end_warehouse])
                        # Check if the load of the path exceeds the maximum allowed load
                        if calculate_load(path, demands) <= max_load
                            push!(paths, path)
                        end
                    end
                end
            end
        end
    end
    return paths
end

function getB1(satellites,routes_1)
    # Initialize a 2D array for b[s][r] with zeros
    b = zeros(Int, length(satellites))
    b = vcat(0 , b)
    i = 2
    while i<=length(routes_1) 
        # println("add line")
        b = hcat(b,zeros(Int, length(satellites)+1))
        i += 1
    end
    # println("satellites= $satellites")
    # println("routes_1= $routes_1")
    # Populate the array
    for (s_idx, i) in enumerate(satellites)  # Loop over points (row index)
        for (r, route) in enumerate(routes_1)  # Loop over routes (column index)
            b[s_idx+1, r] = i in route.sequence ? 1 : 0
        end
    end
    return b
end

function transformRoute(x)
    new_route = Vector{Int}()
    current_node = 0
    if length(x) > (1+length(satellites))^2 
        ## Transform a 2e route
        # Find the starting satellite
        for s in satellites
            for j in A2
                if round(value(x[s, j])) == 1
                    current_node = s
                    push!(new_route, s)
                    break
                end
            end
            if current_node != 0
                break
            end
        end
        while true
            for j in A2
                if round(value(x[current_node,j])) == 1
                    current_node = j
                    push!(new_route,current_node)
                    break
                end
            end
            if current_node in satellites
                break
            end
        end
    else
        ## Transform a 1e route
        current_node = 1 
        push!(new_route, current_node)
        while true
            for j in A1
                if round(value(x[current_node,j])) == 1
                    current_node = j
                    push!(new_route,current_node)
                    break
                end
            end
            if current_node == 1
                break
            end
        end
    end
    
    # generateRoute(new_route)
    # println("transformRoute TEST",new_route)
    return new_route
end

function runMasterProblem(b1)
    ## Relaxed Restricted Master Problem
 
    model = Model(CPLEX.Optimizer)

    # set_silent(model)
    @variable(model, 1>=x[1:length(routes_1)]>=0) # Selection of 1e route
    @variable(model, 1>=y[A2, A2]>=0) # Selection of 2e arc
    @variable(model, capacity_microhub >= z[satellites] >=0) # Amount of freight in satellite s
    @variable(model, f[A2, A2]>=0) # Freight flow
    @variable(model, t[customers]>=0)


    @constraint(model, sync1e[s in satellites], z[s] - M * sum(b1[s,r] * x[r] for r in 1:length(routes_1))<=0)
    @constraint(model, freightOfSatellite[s in satellites], z[s]-sum(f[s,j] for j in customers) == 0)
    @constraint(model, flowConservation2e[i in A2], sum(y[i,j] for j in A2) == sum(y[j,i] for j in A2))
    @constraint(model, flowConservationSatellite[s in satellites], sum(y[s,j] for j in A2) <= capacity_microhub)
    @constraint(model, flowConservationCustomer[c in customers], sum(y[c,j] for j in A2) == 1)
    @constraint(model, customerDemand[c in customers], sum(f[i,c] for i in A2) - sum(f[c,i] for i in A2) == demands[c])
    @constraint(model, capacity2eVehicle[i in A2, j in A2], f[i,j] <= y[i,j] * capacity_2e_vehicle)
    @constraint(model, MTZ[i in customers, j in customers], t[i] + arc_cost[i,j] * y[i,j] <= t[j] + MM * (1-y[i,j]))
    @constraint(model, distanceInitialization[i in satellites, j in customers], arc_cost[i,j] <= t[j]+MM*(1-y[i,j]))
    @constraint(model, distanceEndding[i in customers, j in satellites], t[i] + arc_cost[i,j] <= maximum_duration_2e_vehicle+MM*(1-y[i,j]))
    @constraint(model, distanceLimit[i in customers], t[i]<= maximum_duration_2e_vehicle)

    @objective(model, Min, 
        sum(x[r] * routes_1[r].cost for r in 1:length(routes_1))+sum(y[i,j] * arc_cost[i,j] for i in A2 for j in A2))

    # unset_integer.(x)
    # unset_integer.(y)
    
    optimize!(model)
    π1 = vcat(0, abs.(collect(dual.(sync1e))))
    @assert is_solved_and_feasible(model; dual=true)
    println("objective value of master problem is: ",value(objective_value(model)))
    
    println("Status of 1e route: ")
    # for r in 1:length(routes_1)
    #     println("   x[$r]=",value(x[r]))
    # end
    for r in 1:length(routes_1)
        if value(x[r]) != 0
            println("   1e Route ", routes_1[r].sequence, "   Load= ", routes_1[r].load ,  "   Cost= ", round(routes_1[r].cost,digits=2))
        end 
    end

    println("Status of satellite:")
    for s in satellites
        if value(z[s])!=0
            println("   z[$s]=", value(z[s]))
        end
    end
    println("")
    println("Selection of 2e arc:")
    for i in A2
        for j in A2
            if value(y[i,j])!=0
                println("   y[$i $j]=",value(y[i,j]))
            end
        end
    end
    println("")
    println("Freight flow:")
    for i in A2
        for j in A2
            if value(f[i,j])!=0
                println("   f[$i $j]=",value(f[i,j]))
            end
        end
    end
    println("")
    println("Accumulate distance:")
    for i in customers
        if value(t[i])!=0
            println("   t[$i]=",value(t[i]))
        end
    end
    return π1, value.(y)
end

function solve_subproblem_milp(π)
    println("π1= ",π)
    println("M= $M")
    ## Construct model
    model = Model(CPLEX.Optimizer)

    set_silent(model)
    @variable(model, 1 >= x1[A1,A1] >= 0,Int)
    @variable(model, length(A1)>=u[A1]>=0, Int)

    # @constraint(model, satelliteOblg, sum(x1[i,j] for i in A1 for j in subSatellites)>=1)
    @constraint(model, depart, sum(x1[1,i] for i in satellites) ==1)
    @constraint(model, ending, sum(x1[i,1] for i in satellites) ==1)
    @constraint(model, flow[i in A1], sum(x1[j,i] for j in A1)
                                        ==sum(x1[i,j] for j in A1))
    @constraint(model, subtourElm[i in satellites, j in satellites], 
                u[i] + 1 <= u[j] + MM *(1-x1[i,j]))


    @objective(model, Min, 
        sum(arc_cost[i,j] * x1[i,j] for i in A1 for j in A1) - 
        M * sum(π[s] * x1[s,j] for j in A1 for s in satellites))
    
    # set_optimizer_attribute(model, "CPX_PARAM_TILIM", 120)
    optimize!(model)

    println("Subprobem 1e: Reduced cost= ", objective_value(model))
    # for i in 1:length(A1)
    #     for j in 1:length(A1)
    #         if round(value(x1[i,j]))==1
    #             println("   x1[$i,$j]=",round(value(x1[i,j]),digits=2))
    #         end
    #     end
    # end
    # println("")
    # for i in A1
    #     println("   u[$i]= ",value(u[i]))
    # end
    new_route = nothing
    if objective_value(model) < 0
        # Reduced cost negative, new column generated and added to master problem
        # println("test  : $num_iter")
        new_route = transformRoute(x1) 
    end
    return new_route, objective_value(model)
end

nb_s = 10
nb_m = 4

coor_cust = [[10,35],[10,40],[8,40],[8,45],[5,35]]
demands = vcat(zeros(Int, nb_s + 1) , [5,10,8,5,12])

PI = vcat(ones(nb_m),zeros(nb_s-nb_m))
Random.seed!(42)
shuffle!(PI)
x_coor_parkings = []
y_coor_parkings = []
x_coor_customers = [point[1] for point in coor_cust]
y_coor_customers = [point[2] for point in coor_cust]
min_x,min_y = minimum(x_coor_customers),minimum(y_coor_customers)
max_x,max_y = maximum(x_coor_customers),maximum(y_coor_customers)
for i in 1:nb_s
    push!(x_coor_parkings, rand(min_x: max_x))
    push!(y_coor_parkings, rand(min_y: max_y))
end
coor_parkings = [[x,y] for (x,y) in zip(x_coor_parkings , y_coor_parkings)]
coor = vcat([[0,0]], coor_parkings, coor_cust)
nb_vehicle_per_satellite = 10
capacity_2e_vehicle = 15
capacity_microhub = 50
maximum_duration_2e_vehicle = 30
points = 1:length(demands)

# Define the range of customers
customers = 2+nb_s : length(coor)
satellites = 2:1 + nb_s
arc_cost = calculate_arc_cost(coor)
A2 = 2:length(coor)
A1 = 1:1+length(satellites)
M = sum(demands)# capacity of a satellite
MM = 100

rs_1 =  [[1,10,1],[1,5,1]]
route_1 = []
routes_1 = Vector{Route}()
for r in rs_1
    push!(routes_1, generateRoute(r))
end
rc1 = []
b1 = getB1(satellites, routes_1)

function solve_1e_loop(rc1, routes_1)
    status = false
    num_iter = 1
    y = nothing
    while !(status) 
        println("\n======================Iteration $num_iter======================")
        b1 = getB1(satellites, routes_1)
        π1, y = runMasterProblem(b1)
        
        new_route = nothing
        new_route,rc_v = solve_subproblem_milp(π1)
        if !(isempty(rc1)) && rc_v == rc1[end]
            @info "Reduced cost converged, terminating column generation for 1st echelon"
            status = true
        end
        push!(rc1,rc_v)
        if new_route == nothing
            @info "No new 1e routes generated"
            status = true
        else
            @info "New 1e route $new_route "
            push!(routes_1, generateRoute(new_route))
        end
        num_iter += 1
    end
    return rc1, routes_1, y
end


solve_1e_loop (generic function with 1 method)

In [2]:
π1, y_value = runMasterProblem(b1)
rc1, routes_1, y = solve_1e_loop(rc1, routes_1)

CPLEX Error  3003: Not a mixed-integer problem.
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
Tried aggregator 1 time.
LP Presolve eliminated 182 rows and 164 columns.
Aggregator did 2 substitutions.
Reduced LP has 226 rows, 301 columns, and 997 nonzeros.
Presolve time = 0.00 sec. (0.53 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Dual objective     =             8.650282
objective value of master problem is: 101.2678360144114
Status of 1e route: 
   1e Route [1, 10, 1]   Load= 0   Cost= 89.11
   1e Route [1, 5, 1]   Load= 0   Cost= 74.97
Status of satellite:
   z[5]=35.0
   z[10]=5.0

Selection of 2e arc:
   y[4 12]=0.6666666666666667
   y[4 16]=0.19999999999999996
   y[5 12]=0.3333333333333333
   y[5 13]=0.6666666666666666
   y[5 14]=0.5333333333333333
   y[5 16]=0.8
   y[10 15]=1.0
   y[12 4]=0.8666666666666667
   y[12 5]=0.1333333333333333
   y[13 5]=0.5333333333333333
   y[13 14]=0.4666666666666667
   y[14 5]=0.6666666666666666
   y[14

┌ Info: New 1e route [1, 5, 9, 2, 6, 10, 8, 11, 7, 1] 
└ @ Main /Users/lenovo1/Library/CloudStorage/OneDrive-EcoledeManagementdeNormandie/2EVRPMM/Code/ColumnGeneration/3-variables/semi-compact/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W0sZmlsZQ==.jl:356


Subprobem 1e: Reduced cost= -30.59563025100419

======================Iteration 3======================
CPLEX Error  3003: Not a mixed-integer problem.
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
Tried aggregator 1 time.
LP Presolve eliminated 126 rows and 116 columns.
Aggregator did 4 substitutions.
Reduced LP has 280 rows, 349 columns, and 1185 nonzeros.
Presolve time = 0.00 sec. (0.57 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Dual objective     =             8.650282
Iteration:    62   Dual objective     =            27.068625
Iteration:   124   Dual objective     =            31.887680
objective value of master problem is: 32.128526775979
Status of 1e route: 
   1e Route [1, 5, 3, 11, 8, 10, 6, 2, 4, 1]   Load= 0   Cost= 94.91
Status of satellite:
   z[2]=5.0
   z[3]=5.0
   z[4]=5.0
   z[5]=5.0
   z[6]=5.0
   z[8]=5.0
   z[10]=5.0
   z[11]=5.0

Selection of 2e arc:
   y[2 13]=0.3333333333333333
   y[3 16]=0.3333333333333333
   y[4 

┌ Info: New 1e route [1, 5, 3, 11, 8, 10, 6, 2, 4, 1] 
└ @ Main /Users/lenovo1/Library/CloudStorage/OneDrive-EcoledeManagementdeNormandie/2EVRPMM/Code/ColumnGeneration/3-variables/semi-compact/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W0sZmlsZQ==.jl:356


Subprobem 1e: Reduced cost= -19.831453330086728

======================Iteration 4======================
CPLEX Error  3003: Not a mixed-integer problem.
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
Tried aggregator 1 time.
LP Presolve eliminated 126 rows and 116 columns.
Reduced LP has 284 rows, 354 columns, and 1201 nonzeros.
Presolve time = 0.00 sec. (0.55 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Dual objective     =             8.650282
Iteration:    62   Dual objective     =            27.068625
Iteration:   124   Dual objective     =            31.887680
objective value of master problem is: 31.9839504059277
Status of 1e route: 
   1e Route [1, 5, 3, 11, 8, 10, 6, 2, 4, 1]   Load= 0   Cost= 94.91
   1e Route [1, 5, 3, 11, 7, 10, 9, 2, 4, 1]   Load= 0   Cost= 101.52
Status of satellite:
   z[2]=5.0
   z[3]=5.0
   z[4]=5.0
   z[5]=5.0
   z[6]=4.0
   z[7]=1.0000000000000004
   z[8]=4.0
   z[9]=1.0000000000000004
   z[10]=5.0
   z[11]

...

Subprobem 1e: Reduced cost= -3.2759281427957028

======================Iteration 6======================
CPLEX Error  3003: Not a mixed-integer problem.
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
Tried aggregator 1 time.
LP Presolve eliminated 126 rows and 116 columns.
Reduced LP has 284 rows, 356 columns, and 1217 nonzeros.
Presolve time = 0.00 sec. (0.55 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Dual objective     =             8.650282
Iteration:    62   Dual objective     =            27.068625
objective value of master problem is: 31.767937108023204
Status of 1e route: 
   1e Route [1, 5, 3, 11, 8, 10, 6, 2, 4, 1]   Load= 0   Cost= 94.91
   1e Route [1, 5, 3, 11, 9, 10, 6, 2, 4, 1]   Load= 0   Cost= 100.91
Status of satellite:
   z[2]=5.0
   z[3]=5.0
   z[4]=5.0
   z[5]=5.0
   z[6]=5.0
   z[8]=3.0000000000000004
   z[9]=1.9999999999999996
   z[10]=5.0
   z[11]=5.0

Selection of 2e arc:
   y[2 13]=0.3333333333333333
   y[3 16]=0.3

┌ Info: New 1e route [1, 5, 3, 11, 7, 10, 6, 2, 4, 1] 
└ @ Main /Users/lenovo1/Library/CloudStorage/OneDrive-EcoledeManagementdeNormandie/2EVRPMM/Code/ColumnGeneration/3-variables/semi-compact/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W0sZmlsZQ==.jl:356


Subprobem 1e: Reduced cost= 3.552713678800501e-14


┌ Info: No new 1e routes generated
└ @ Main /Users/lenovo1/Library/CloudStorage/OneDrive-EcoledeManagementdeNormandie/2EVRPMM/Code/ColumnGeneration/3-variables/semi-compact/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W0sZmlsZQ==.jl:353


(Any[-625.4176160814552, -30.59563025100419, -19.831453330086728, -4.320265958089925, -3.2759281427957028, 3.552713678800501e-14], Route[Route(89.10667763978185, [1, 10, 1], 3, 0), Route(74.96665925596525, [1, 5, 1], 3, 0), Route(94.1444967575074, [1, 5, 9, 2, 6, 10, 8, 11, 7, 1], 10, 0), Route(94.90742703330093, [1, 5, 3, 11, 8, 10, 6, 2, 4, 1], 10, 0), Route(101.51853653862311, [1, 5, 3, 11, 7, 10, 9, 2, 4, 1], 10, 0), Route(100.90742703330093, [1, 5, 3, 11, 9, 10, 6, 2, 4, 1], 10, 0), Route(97.63149889050523, [1, 5, 3, 11, 7, 10, 6, 2, 4, 1], 10, 0)], 2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 2:16
    Dimension 2, 2:16
And data, a 15×15 Matrix{Float64}:
 0.0                 0.0                 0.0  …  0.0  0.0
 0.0                 0.0                 0.0     0.0  0.3333333333333333
 0.0                 0.0                 0.0     0.0  0.0
 0.0                 0.0                 0.0     0.0  0.5333333333333334
 0.0                 0.0             

In [5]:
function solve_left_rlmp_model(i_branch, j_branch)
    b1 = getB1(satellites, routes_1)
    model = Model(CPLEX.Optimizer)

    # set_silent(model)
    @variable(model, 1>=x[1:length(routes_1)]>=0) # Selection of 1e route
    @variable(model, 1>=y[A2, A2]>=0) # Selection of 2e arc
    @variable(model, capacity_microhub >= z[satellites] >=0) # Amount of freight in satellite s
    @variable(model, f[A2, A2]>=0) # Freight flow
    @variable(model, t[customers]>=0)


    @constraint(model, sync1e[s in satellites], z[s] - M * sum(b1[s,r] * x[r] for r in 1:length(routes_1))<=0)
    @constraint(model, freightOfSatellite[s in satellites], z[s]-sum(f[s,j] for j in customers) == 0)
    @constraint(model, flowConservation2e[i in A2], sum(y[i,j] for j in A2) == sum(y[j,i] for j in A2))
    @constraint(model, flowConservationSatellite[s in satellites], sum(y[s,j] for j in A2) <= capacity_microhub)
    @constraint(model, flowConservationCustomer[c in customers], sum(y[c,j] for j in A2) == 1)
    @constraint(model, customerDemand[c in customers], sum(f[i,c] for i in A2) - sum(f[c,i] for i in A2) == demands[c])
    @constraint(model, capacity2eVehicle[i in A2, j in A2], f[i,j] <= y[i,j] * capacity_2e_vehicle)
    @constraint(model, MTZ[i in customers, j in customers], t[i] + arc_cost[i,j] * y[i,j] <= t[j] + MM * (1-y[i,j]))
    @constraint(model, distanceInitialization[i in satellites, j in customers], arc_cost[i,j] <= t[j]+MM*(1-y[i,j]))
    @constraint(model, distanceEndding[i in customers, j in satellites], t[i] + arc_cost[i,j] <= maximum_duration_2e_vehicle+MM*(1-y[i,j]))
    @constraint(model, distanceLimit[i in customers], t[i]<= maximum_duration_2e_vehicle)
    @constraint(model, leftModel, y[i_branch, j_branch] == 1)

    @objective(model, Min, 
        sum(x[r] * routes_1[r].cost for r in 1:length(routes_1))+sum(y[i,j] * arc_cost[i,j] for i in A2 for j in A2))

    optimize!(model)
    
    println("Selection of 2e arc:")
    for i in A2
        for j in A2
            if value(y[i,j])!=0
                println("   y[$i $j]=",value(y[i,j]))
            end
        end
    end

end

function branch_on_variables(y)
    fractional_y = [(i,j) for i in A2, j in A2 if 0<y_value[i,j]<1]
    if isempty(fractional_y)
        println("Optimal Integer Solution Found")
    else
        (i_branch, j_branch) = fractional_y[1]
    end
    println("Branching on y[$i_branch, $j_branch]")

    solve_left_rlmp_model(i_branch, j_branch)
end


# When no more new routes generated, start branching
branch_on_variables(y)


Branching on y[12, 4]
CPLEX Error  3003: Not a mixed-integer problem.
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
Tried aggregator 1 time.
LP Presolve eliminated 166 rows and 145 columns.
Reduced LP has 245 rows, 327 columns, and 1074 nonzeros.
Presolve time = 0.00 sec. (0.52 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Dual objective     =             8.650282
Iteration:    62   Dual objective     =            27.068625
Selection of 2e arc:
   y[2 13]=0.3333333333333336
   y[3 16]=0.33333333333333326
   y[4 12]=1.0
   y[5 16]=0.5333333333333332
   y[6 14]=0.3333333333333334
   y[8 11]=0.3333333333333333
   y[8 13]=0.19999999999999968
   y[9 16]=0.1333333333333335
   y[10 15]=1.0
   y[11 13]=0.3333333333333333
   y[12 4]=1.0
   y[13 2]=0.3333333333333336
   y[13 14]=0.6666666666666664
   y[14 6]=0.3333333333333334
   y[14 8]=0.533333333333333
   y[14 13]=0.13333333333333353
   y[15 10]=1.0
   y[16 3]=0.33333333333333326
   y[16 5]=0.53333

In [ ]:
nb_s = 10
nb_m = 4

coor_cust = [[10,35],[10,40],[8,40],[8,45],[5,35]]
demands = vcat(zeros(Int, nb_s + 1) , [5,10,8,5,12])

PI = vcat(ones(nb_m),zeros(nb_s-nb_m))
Random.seed!(42)
shuffle!(PI)
x_coor_parkings = []
y_coor_parkings = []
x_coor_customers = [point[1] for point in coor_cust]
y_coor_customers = [point[2] for point in coor_cust]
min_x,min_y = minimum(x_coor_customers),minimum(y_coor_customers)
max_x,max_y = maximum(x_coor_customers),maximum(y_coor_customers)
for i in 1:nb_s
    push!(x_coor_parkings, rand(min_x: max_x))
    push!(y_coor_parkings, rand(min_y: max_y))
end
coor_parkings = [[x,y] for (x,y) in zip(x_coor_parkings , y_coor_parkings)]
coor = vcat([[0,0]], coor_parkings, coor_cust)
nb_vehicle_per_satellite = 10
capacity_2e_vehicle = 15
capacity_microhub = 50
maximum_duration_2e_vehicle = 30
points = 1:length(demands)

# Define the range of customers
customers = 2+nb_s : length(coor)
satellites = 2:1 + nb_s
arc_cost = calculate_arc_cost(coor)
A2 = 2:length(coor)
A1 = 1:1+length(satellites)
M = sum(demands)# capacity of a satellite
MM = 100



model = Model(CPLEX.Optimizer)

# set_silent(model)
@variable(model, 1>=x[1:length(routes_1)]>=0) # Selection of 1e route
@variable(model, 1>=y[A2, A2]>=0) # Selection of 2e arc
@variable(model, capacity_microhub >= z[satellites] >=0) # Amount of freight in satellite s
@variable(model, f[A2, A2]>=0) # Freight flow
@variable(model, t[customers]>=0)


@constraint(model, sync1e[s in satellites], z[s] - M * sum(b1[s,r] * x[r] for r in 1:length(routes_1))<=0)
@constraint(model, freightOfSatellite[s in satellites], z[s]-sum(f[s,j] for j in customers) == 0)
@constraint(model, flowConservation2e[i in A2], sum(y[i,j] for j in A2) == sum(y[j,i] for j in A2))
@constraint(model, flowConservationSatellite[s in satellites], sum(y[s,j] for j in A2) <= capacity_microhub)
@constraint(model, flowConservationCustomer[c in customers], sum(y[c,j] for j in A2) == 1)
@constraint(model, customerDemand[c in customers], sum(f[i,c] for i in A2) - sum(f[c,i] for i in A2) == demands[c])
@constraint(model, capacity2eVehicle[i in A2, j in A2], f[i,j] <= y[i,j] * capacity_2e_vehicle)
@constraint(model, MTZ[i in customers, j in customers], t[i] + arc_cost[i,j] * y[i,j] <= t[j] + MM * (1-y[i,j]))
@constraint(model, distanceInitialization[i in satellites, j in customers], arc_cost[i,j] <= t[j]+MM*(1-y[i,j]))
@constraint(model, distanceEndding[i in customers, j in satellites], t[i] + arc_cost[i,j] <= maximum_duration_2e_vehicle+MM*(1-y[i,j]))
@constraint(model, distanceLimit[i in customers], t[i]<= maximum_duration_2e_vehicle)

@objective(model, Min, 
    sum(x[r] * routes_1[r].cost for r in 1:length(routes_1))+sum(y[i,j] * arc_cost[i,j] for i in A2 for j in A2))

# optimize!(model)

CPLEX Error  3003: Not a mixed-integer problem.
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
Tried aggregator 1 time.
LP Presolve eliminated 182 rows and 164 columns.
Aggregator did 2 substitutions.
Reduced LP has 226 rows, 301 columns, and 997 nonzeros.
Presolve time = 0.00 sec. (0.53 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Dual objective     =             8.650282


In [ ]:
rs_1 =  [[1,10,1],[1,5,1]]
route_1 = []
routes_1 = Vector{Route}()
for r in rs_1
    push!(routes_1, generateRoute(r))
end
rc1 = []
num_iter = 1

while true 
    println("\n======================Iteration $num_iter======================")
    b1 = getB1(satellites, routes_1)
    println(b1)
    println("length of 1e routes",length(routes_1))
    optimize!(model)
    π1 = vcat(0, abs.(collect(dual.(sync1e))))
    println(π1)
    println("objective value of master problem is: ",value(objective_value(model)))
    
    new_route = nothing
    new_route,rc_v = solve_subproblem_milp(π1)
    println("new route= $new_route")
    if !(isempty(rc1)) && rc_v == rc1[end]
        @info "Reduced cost converged, terminating column generation for 1st echelon"
        status = break
    end
    push!(rc1,rc_v)
    if new_route == nothing
        @info "No new 1e routes generated"
        status = break
    else
        @info "New 1e route $new_route "
        push!(routes_1, generateRoute(new_route))
    end
    num_iter += 1
end

num_iteration = 1
while true && num_iteration < 5
    ## Column Generation



    ## Branch


    num_iteration += 1
end


======================Iteration 1======================
[0 0 0 0 0; 0 0 1 1 1; 0 0 0 0 0; 0 0 0 0 0; 0 1 1 1 1; 0 0 1 1 1; 0 0 1 1 1; 0 0 1 1 1; 0 0 1 1 1; 1 0 1 1 1; 0 0 1 1 1]
length of 1e routes5
CPLEX Error  3003: Not a mixed-integer problem.
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
Using devex.
[0.0, 2.314536566430064, 2.314536566430064, 2.314536566430064, 1.8741664813991314, 2.314536566430064, 2.314536566430064, 2.314536566430064, 2.314536566430064, 2.2276669409945464, 2.314536566430064]
objective value of master problem is: 101.26783601441137
π1= [0.0, 2.314536566430064, 2.314536566430064, 2.314536566430064, 1.8741664813991314, 2.314536566430064, 2.314536566430064, 2.314536566430064, 2.314536566430064, 2.2276669409945464, 2.314536566430064]
M= 40
